In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as MF
import dgl
import dgl.nn as dglnn
from dgl.data import AsNodePredDataset
from dgl.dataloading import DataLoader, NeighborSampler, MultiLayerFullNeighborSampler
from ogb.nodeproppred import DglNodePropPredDataset
import tqdm
import argparse
import ast
import sklearn.metrics
import numpy as np
import time
import sys
from dgl.dataloading import (
    as_edge_prediction_sampler,
    DataLoader,
    MultiLayerFullNeighborSampler,
    negative_sampler,
    NeighborSampler,
)

In [ ]:
dataset = AsNodePredDataset(DglNodePropPredDataset('ogbn-products',root="/home/bear/workspace/singleGNN/data/dataset"))
g = dataset[0]

Products数据集
nodeNUM:2449029
edgeNUM:123718152
feat:100
class:47
trainNUM:196615
valNUM:39323
testNUM:2213091

In [ ]:
import random
edgeNUM = 1000000
sampled_edge_ids = random.sample(range(g.num_edges()), 1000000)
trainId=sampled_edge_ids[:int(edgeNUM * 0.3)]
TestId=sampled_edge_ids[int(edgeNUM * 0.3):int(edgeNUM * 0.9)]
ValId=sampled_edge_ids[int(edgeNUM * 0.9):]


neg_train_sampler = dgl.dataloading.negative_sampler.Uniform(1)
train_src, train_neg_dst = neg_train_sampler(g, torch.Tensor(trainId).to(torch.int64))
raw_train_dst=torch.Tensor(g.edges()[1][trainId])

neg_val_sampler = dgl.dataloading.negative_sampler.Uniform(1000)
val_src,val_neg_dst = neg_val_sampler(g, torch.Tensor(ValId).to(torch.int64))
raw_val_dst=torch.Tensor(g.edges()[1][ValId])

neg_test_sampler = dgl.dataloading.negative_sampler.Uniform(1000)
test_src, test_neg_dst = neg_test_sampler(g, torch.Tensor(TestId).to(torch.int64))
raw_test_dst=torch.Tensor(g.edges()[1][TestId])